In [4]:
import json
import carla
import math
import random
import threading
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent

def leggi_json_da_file(nome_file):
    json_list = []  # Lista per accumulare i JSON
    try:
        with open(nome_file, "r", encoding="utf-8") as file:
            for i, riga in enumerate(file, start=1):
                riga = riga.strip()  # Rimuove spazi vuoti o caratteri di fine linea
                if riga:  # Verifica che la riga non sia vuota
                    try:
                        json_obj = json.loads(riga)  # Converte la riga in oggetto JSON
                        json_list.append(json_obj)
                    except json.JSONDecodeError as e:
                        print(f"Errore nella riga {i}: formato JSON non valido ({e})")
        return json_list
    except FileNotFoundError:
        print(f"Il file '{nome_file}' non esiste.")
    except Exception as e:
        print(f"Errore durante la lettura del file: {e}")
    return []


################################################################ AUX ########################################################################################
#Funzione ausiliaria per porre il focus spettatore dall'alto
def focus_above(world, loc, z_height=60, in_pitch=-90):
    camera_location = carla.Location(loc.x, loc.y, z=z_height) #altezza 60
    camera_rotation = carla.Rotation(pitch=in_pitch)  # Orientamento verso il basso
    spectator = world.get_spectator()
    spectator.set_transform(carla.Transform(camera_location, camera_rotation))

# shared event to force thread stop
stop_event = threading.Event()

def stop_threads():
    stop_event.set()  # set stop thread flag

def reset_stop_event():
    stop_event.clear()
	
#calcola la distanza in metri tra due wp
def wp_distance(wp1, wp2):
    distance = 999999
    if wp1 is not None and wp2 is not None:
        distance =  wp1.transform.location.distance(wp2.transform.location)
    return abs(distance)

#routine that starts a thread for each pilot agent (agent behaviour can be "cautious", "normal", "aggressive")
def set_agent_vehicle(world, in_vehicle, dest_wp, agent_behaviour='normal', disable_trf = False):
    
    #wp of vehicle
    start_location = in_vehicle.get_location()
    start_waypoint = world.get_map().get_waypoint(start_location)
    
    #AGENT REACTS TO TRAFFIC LIGHTS, STOP, TRIES TO AVOID COLLISION WITH PEDESTRIANS AND VEHICLES.. 
    #3 PROFILES AL MOMENTO: cautious, normal, aggressive -> https://carla.readthedocs.io/en/0.9.12/adv_agents/#behavior-types
    agent = BehaviorAgent(in_vehicle, behavior=agent_behaviour)
    
    #skip sem if disable_trf=True
    agent.ignore_traffic_lights(disable_trf)
    
    agent.set_destination(dest_wp.transform.location)
    return agent

def start_vehicle(world, spawned_vehicle, symbol, visible = 1, behaviour = 'normal', disable_trf = False):
    thread = None
    print(f"in start {spawned_vehicle}")
    v, destwp= spawned_vehicle
    if destwp is not None and destwp is not None:
        thread = threading.Thread(target=roam_vehicle, args=(world, v, destwp, behaviour.lower(), disable_trf))
        thread.start()
    return thread;

################################################################ AUX ########################################################################################

# Esempio di utilizzo
nome_file = "sim_data_all.log"
json_array = leggi_json_da_file(nome_file)

client = carla.Client('localhost', 2000)
client.set_timeout(5.0)
world = client.get_world()
bp_library = world.get_blueprint_library() 
map = world.get_map()

#iterating file
for j in json_array:
    
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sen in world.get_actors().filter('*sensor*'):
        sen.destroy()
    
    ego_spawn = j['ego_spawn']
    npc_spawn = j['npc_spawn']
    
    ego_dest = j['ego_dest']
    npc_dest = j['npc_dest']
    
    print(j)
    
    leading_car_bp = bp_library.find('vehicle.lincoln.mkz_2020')
    # per ora a mano ma da prendere via json
    lead_transform = carla.Transform(carla.Location(x=ego_spawn['x'], y=ego_spawn['y'], z=ego_spawn['z']), carla.Rotation(pitch=ego_spawn['pitch'], yaw=ego_spawn['yaw'], roll=ego_spawn['roll']))
    leading_car = world.spawn_actor(leading_car_bp, lead_transform)
    
    print(leading_car)
    
    lagging_car_bp = bp_library.find('vehicle.audi.tt')
    #lag_transform = carla.Transform(carla.Location(x=391.5, y=10.0, z=0.02), carla.Rotation(yaw=89.6))
    lag_transform = carla.Transform(carla.Location(x=npc_spawn['x'], y=npc_spawn['y'], z=npc_spawn['z']), carla.Rotation(pitch=npc_spawn['pitch'], yaw=npc_spawn['yaw'], roll=npc_spawn['roll']))
    lagging_car = world.try_spawn_actor(lagging_car_bp, lag_transform)

    #focus camera
    focus_above(world, lag_transform.location, 60, -90)
    
    carla_map = world.get_map()
    dest_lag_wp = carla_map.get_waypoint( carla.Location(x=ego_dest['x'], y=ego_dest['y'], z=ego_dest['z']) )
    dest_lead_wp = carla_map.get_waypoint( carla.Location(x=npc_dest['x'], y=npc_dest['y'], z=npc_dest['z']) )
    
    agent_lag = set_agent_vehicle(world, lagging_car, dest_lag_wp, 'aggressive', disable_trf = True)
    agent_lead = set_agent_vehicle(world, leading_car, dest_lead_wp, 'cautious', disable_trf = True)
    
    print(lagging_car)
    
    print("Started simulation")
    
    #main agent while
    while True:
        #for each iteration applies agent controls for leading and lagging cars
        if not agent_lag.done() and lagging_car.is_alive:
           lagging_car.apply_control(agent_lag.run_step())
        if not agent_lead.done() and leading_car.is_alive:
           leading_car.apply_control(agent_lead.run_step())
           
        curr_lag_wp = world.get_map().get_waypoint(lagging_car.get_location())
        curr_lead_wp = world.get_map().get_waypoint(lagging_car.get_location())
    
        if agent_lead.done() or wp_distance(curr_lead_wp, dest_lead_wp) <= 3.7:
            if leading_car.is_alive:
               leading_car.destroy()
        
        if agent_lag.done() or wp_distance(curr_lag_wp, dest_lag_wp) <= 3.7:
            if leading_car.is_alive:
               leading_car.destroy()
            lagging_car.destroy()
            print(f"Agent reached destination")
            break
    
    print("End simulation")

print("PROGRAM ENDED")


{'junction_id': 8, 'ego_spawn': {'x': 17.091217041015625, 'y': 28.104215621948242, 'z': 0.5999999642372131, 'pitch': 0.0, 'yaw': 0.15919798612594604, 'roll': 0.0}, 'ego_dest': {'x': 57.40364456176758, 'y': 28.34353256225586, 'z': 0.0, 'pitch': 0.0, 'yaw': -359.8407897949219, 'roll': 0.0}, 'npc_spawn': {'x': 44.02894973754883, 'y': 52.54869842529297, 'z': 0.5999999642372131, 'pitch': 0.0, 'yaw': -90.75109100341797, 'roll': 0.0}, 'npc_dest': {'x': 57.40364456176758, 'y': 28.34353256225586, 'z': 0.0, 'pitch': 0.0, 'yaw': 0, 'roll': 0.0}, 'ego_agent': 'aggressive', 'traffic_agent': 'aggressive'}
Actor(id=320, type=vehicle.lincoln.mkz_2020)
Actor(id=321, type=vehicle.audi.tt)
Started simulation
Agent reached destination
End simulation
{'junction_id': 8, 'ego_spawn': {'x': 44.02894973754883, 'y': 52.54869842529297, 'z': 0.5999999642372131, 'pitch': 0.0, 'yaw': -90.75109100341797, 'roll': 0.0}, 'ego_dest': {'x': 57.40364456176758, 'y': 28.34353256225586, 'z': 0.0, 'pitch': 0.0, 'yaw': 0.15919